# Защита персональных данных клиентов

# Содержание

- [Введение](#intro)
- [Загрузка данных](#data)
- [Умножение матриц](#multiplication)
- [Алгоритм преобразования](#algorithm)
- [Проверка алгоритма](#testing)
- [Вывод](#conclusion)

<a id='intro'></a>
## Введение.

Перед нами стоит задачи разработать алгоритм преобразования данных для страховой компании с целью их шифрования, чтобы по ним было сложно восстановить персональную информацию. Качество модели при этом не должно быть хуже чем с оригинальными данными. Нам предоставлен файл содержащий данные клиентов страховой компании, изучим его, если надо сделаем предобработку и будем использовать эти данные для тестирования алгоритма.

<a id='data'></a>
## Загрузка данных

**Импортируем необходимые библиотеки**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline

**Читаем исходный файл**

In [2]:
try:
    data = pd.read_csv('insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')

**Основная информация о датафрейме**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


В датафреме 5000 строк. Типы данных в колонках `Возраст` и `Зарплата` находятся в вещественном типе, преобразуем их в целочисленный.

**Изменение типов данных**

In [5]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int32
 2   Зарплата           5000 non-null   int32
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int32(2), int64(3)
memory usage: 156.4 KB


In [7]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


Типы заменили, данные готовы для работы с ними. Можем приступать к следующему шагу.

<a id='multiplication'></a>
## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении признаков на обратимую матрицу качество линейной регресии не изменится.

**Обоснование:** При умножении матрицы $X$ на матрицу $P$ получаем новую матрицу, пусть это будет матрица $Z$:

$$
Z=XP
$$

Качество регресии будет совпадать если $a$ не изменнённой матрицы будет совпадать с $a_P$ - предсказания с ипользованием матрицы $Z$.

$$
a=a_P
$$

Посмотрим на $a_P$ и докажем что будет совпадение с $a$.
Так как $a = Xw$ и $w = (X^T X)^{-1} X^T y$ получаем:

$$
a_P=Zw_P
$$
$$
w_P=(Z^T Z)^{-1} Z^T y
$$


Для дальнейших расчётов $a_P$, заменим $Z$ на $XP$:
$$
a_P=XPw_P
$$
$$
w_P=((XP)^T XP)^{-1} (XP)^T y
$$

В раскрытии $w_P$ нам поможет следущее правило: $(ABС)^{-1} = ((AB)С)^{-1} = ((AB)С)^{-1} = С^{-1}(AB)^{-1} = С^{-1}B^{-1}A^{-1}$

$$
w_P=P^{-1}((XP)^{T}X)^{-1} (XP)^Ty
$$

Вспомним одно из свойств транспонированной матрицы. Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке. $(AB)^T=B^TA^T$.

$$
w_P=P^{-1}((XP)^{T}X)^{-1} P^TX^Ty
$$
$$
w_P=P^{-1}(P^TX^TX)^{-1} P^TX^Ty
$$

Применив вышеуказанное правило получили из $((XP)^T XP)^{-1}$ три матрицы $P^{-1}(X^TX)^{-1}(P^T)^{-1}$, теперь $w_P$ выглядит следущим образом:

$$
w_P=P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty
$$

$(P^T)^{-1} P$ будет равно  $E$ тогда:

$$
w_P=P^{-1}(X^TX)^{-1} X^Ty
$$

Из этого следует:

$$
w_P=P^{-1}w
$$

Посмотрим на $a_P$:
$$
a_P=Zw_P= XPP^{-1}w
$$

$PP^{-1}$ как и $(P^T)^{-1} P$ ранее будет равно  $E$ тогда:

$$
a_P=Xw=a
$$

Мы доказали что при умножении признаков на обратимую матрицу значение предсказаний не меняются $a=a_P$, а это значит качество линейной регресии не поменяется. 

<a id='algorithm'></a>
## Алгоритм преобразования

**Алгоритм**

Будем производить умножение матрицы признаков на случайно сгенерированную обратимую матрицу.

- Сначала сгенерируем обратимую матрицу $P$
- Сделаем проверку на обратимость
- После того как убедимся что матрица обратимая, получим матрицу $Z$ перемножив матрицы $X$ и $P$
- Матрица $Z$ будет являтся матрицей с зашифрованными признаками. Эту матрицу будем использовать для обучения модели.

**Обоснование**

В предудущем пункте мы доказали что при умножении матрицы признаков на случайную обратимую матрицу качество линейной регресии не поменяется. При генерации матрицы $P$ нужно обратить внимание на то чтобы определитель не был равен нулю (матрица обратима только когда она невырожденная). Матрица $P$ квадратная $(𝑛×𝑛)$, при генерации нужно учёсть её размер. Матрица $X$ имеет размер $(𝑚×𝑛)$. $𝑛$ этих матриц должен совпадать, тогда мы получим матрицу $Z$ такого же размера как и $X$. 

<a id='testing'></a>
## Проверка алгоритма

**Выделение целевого признака**

Перед тем как приступать к обучению модели выделим целевой признак, он находится в колонке `Страховые выплаты`.

In [8]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

**Разбитие на выборки**

Сделаем разбитие данных на две выборки:

- Тренировочную - для обучения модели
- Валидационную - на которой мы будем проверять модель

Разделение произведём в соотношении 3:1, тоесть 75% - тренировочная выборка, 25% - валидационная выборка.

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [10]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(3750, 4)
(1250, 4)
(3750,)
(1250,)


Сделали разбитие, проверили размеры - всё корректно.

**Прогноз на незашифрованных данных**

Обучим модель линейной регресии на назашифрованных данных, сделаем предсказание и узнаем качество модели по метрике качества R2. Данным показатель в дальнейшем должен совпасть с предсказаниями сделанными на зашифрованном датасете.

In [11]:
model = LinearRegression().fit(features_train, target_train)

In [12]:
predictions = model.predict(features_test)
print('R2 =', r2_score(target_test, predictions))

R2 = 0.4352275684083253


Получили **R2: 0.435**

**Создание зашифрованного датасета**

Напишем функцию с помощью которой получим зашифрованные данные. Основной принцып алгоритма был описан выше. Создадим случайную квадратную матрицу у которой размер будет равен ширине матрицы с признаками. Определитель должен быть не равен нулю, поэтому при генерации добавим проверку определителя, если он окажентся 0, тогда генерируюм другую матрицу, пока не получим нужную.

In [13]:
def change_features(features):
    n = features.shape[1]
    matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(matrix)
    while det == 0:
        matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(matrix)
    changed_features = features @ matrix
    return changed_features

In [14]:
new_features = change_features(features)
display(new_features.head())
print(new_features.shape)

,0,1,2,3
0,99372,198617,148978,148885
1,76189,152239,114190,114093
2,42116,84145,63116,63058
3,83494,166923,125196,125144
4,52315,104543,78420,78358


(5000, 4)


Создали зашифрованные признаки и проверили размер датасета. Размер соотвествует оригинальному датасету. Теперь произведём разбитие на выборки новых признаков, в том же соотношении как и прежде (75%/25%).

In [15]:
new_features_train, new_features_test, new_target_train, new_target_test = train_test_split(
    new_features, target, test_size=0.25, random_state=12345)

In [16]:
print(new_features_train.shape)
print(new_features_test.shape)
print(new_target_train.shape)
print(new_target_test.shape)

(3750, 4)
(1250, 4)
(3750,)
(1250,)


Сделали разбитие, проверили размеры - всё корректно.

**Прогноз на зашифрованных данных**

Обучим модель на новых признаках, сделаем прогноз и проверим соотвествие качества модели с предыдущей.

In [17]:
crypted_model = LinearRegression().fit(new_features_train, new_target_train)

In [18]:
new_predictions = crypted_model.predict(new_features_test)
print('R2 =', r2_score(new_target_test, new_predictions))

R2 = 0.43522756840844057


Качество модели совпадает **R2: 0.435**.

<a id='conclusion'></a>
## Вывод

Целью данного проекта являлось разработка метода преобразования данных при котором качество модели машинного обучения не изменится и изначальные данные будет трудно восстановить.

Предложили алгоритм при котором матрицу содержащую признаки умножают на случайно сгенерированную обратимую матрицу. Тем самым мы зашифруем оригинальные значения. Качество модели при этом не ухудшится, мы доказали это с помощью детального разбора формул.

Реализовали данный алгоритм. Построили две модели линейной регрессии, одна с оригинальными признаки, вторая с зашифрованными. Сравнили их качество, качество не изменилось, в обоих случаях метрика качества **R2 = 0.435**. Можем сделать вывод что такой метод шифровки данных корректный и имеет право на использование.